<a href="https://colab.research.google.com/github/Vakhranev/Pushkina/blob/main/%D0%A1%D0%BC%D1%91%D1%80%D0%B4%D0%B6%D0%B5%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%86%D1%8B%20%D0%B8%20%D0%9C%D0%BE%D0%BD%D0%B0%D1%85%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Загрузка файла
df = pd.read_excel("Термины 5-11 классы Монахов (1).xlsx")

# Таблица соответствий
mapping = {
    "artisticculture": "Изобразительное искусство",
    "art": "Изобразительное искусство",
    "geometry": "Геометрия",
    "maths": "Математика",
    "matan": "Алгебра",
    "algebra": "Алгебра",
    "biology": "Биология",
    "natsci": "Естествознание",
    "literature": "Литература",
    "physicaled": "Физическая культура",
    "informatics": "Информатика",
    "music": "Музыка",
    "astronomy": "Астрономия",
    "crafts": "Технология",
    "ruslang": "Русский язык",
    "sociology": "Обществознание",
    "physics": "Физика",
    "law": "Обществознание",
    "chemistry": "Химия",
    "geography": "География",
    "history": "История"
}

# Функция обработки дисциплин и классов
def process_disciplines(disciplines, stages):
    if isinstance(disciplines, str):
        disciplines = eval(disciplines)  # Преобразуем строку в список
    elif isinstance(disciplines, float):  # Проверяем, является ли disciplines числом с плавающей точкой
        disciplines = [disciplines]     # Если да, то преобразуем его в список
    if isinstance(stages, str):
        stages = eval(stages)
    elif isinstance(stages, float):  # Проверяем, является ли stages числом с плавающей точкой
        stages = [stages]       # Если да, то преобразуем его в список

    new_disciplines = []
    new_stages = []
    seen = set()

    for i, subject in enumerate(disciplines):
        mapped_subject = mapping.get(subject, subject)
        if mapped_subject not in seen:
            seen.add(mapped_subject)
            new_disciplines.append(mapped_subject)
            new_stages.append(stages[i])

    return str(new_disciplines), str(new_stages)

# Применение обработки
df[['discipline', 'stages']] = df.apply(lambda row: process_disciplines(row['discipline'], row['stages']), axis=1, result_type='expand')

# Сохранение результата
df.to_excel("Термины 5-11 классы Монахов (обработанный).xlsx", index=False)

print("Файл успешно обработан и сохранён!")

Файл успешно обработан и сохранён!


In [ ]:
!pip install pymorphy3

import pandas as pd
import pymorphy3
import re

In [5]:
import pandas as pd
import pymorphy3
import re

# Загружаем файл
df = pd.read_excel("terms (6).xlsx")

# Инициализируем морфологический анализатор
morph = pymorphy3.MorphAnalyzer()

def normalize_phrase(phrase):
    words = re.sub(r',', '', phrase).split()
    normalized_words = [morph.parse(word)[0].normal_form for word in words]
    return "_".join(normalized_words)

# Создаём новый столбец с нормализованными терминами
df["F"] = df["Слово или сочетание"].astype(str).apply(normalize_phrase)

# Обрабатываем дублированные строки
processed_rows = {}

for _, row in df.iterrows():
    term = row["F"]
    subjects = row["Предмет"].split(", ") if pd.notna(row["Предмет"]) else []
    min_classes = list(map(int, str(row["Минимальный класс, где встречается"]).split(", "))) if pd.notna(row["Минимальный класс, где встречается"]) else []
    freq = row["Частота"]

    if term in processed_rows:
        existing = processed_rows[term]

        # Создаём словари предметов и классов
        existing_subjects = existing["Предмет"].split(", ") if pd.notna(existing["Предмет"]) else []
        existing_classes = list(map(int, str(existing["Минимальный класс, где встречается"]).split(", "))) if pd.notna(existing["Минимальный класс, где встречается"]) else []

        subject_class_map = dict(zip(existing_subjects, existing_classes))

        for subj, cls in zip(subjects, min_classes):
            if subj in subject_class_map:
                subject_class_map[subj] = min(subject_class_map[subj], cls)
            else:
                subject_class_map[subj] = cls

        # Обновляем данные
        existing["Предмет"] = ", ".join(subject_class_map.keys())
        existing["Минимальный класс, где встречается"] = ", ".join(map(str, subject_class_map.values()))
        existing["Частота"] += freq

    else:
        processed_rows[term] = row.to_dict()

# Преобразуем обратно в DataFrame
result_df = pd.DataFrame(processed_rows.values())

# Сохраняем результат
output_file = "terms_normalized.xlsx"
result_df.to_excel(output_file, index=False)
print(f"Файл {output_file} успешно сохранён!")

Файл terms_normalized.xlsx успешно сохранён!


In [6]:
import pandas as pd

# Загружаем оба файла
terms_df = pd.read_excel("terms_normalized.xlsx")
monakhov_df = pd.read_excel("Термины 5-11 классы Монахов (обработанный).xlsx")

# Объединяем по Начальным формам и termin
merged_df = pd.merge(terms_df, monakhov_df, left_on="F", right_on="termin", how="outer")

# Сохраняем в новый файл
merged_df.to_excel("merged_terms.xlsx", index=False)

print("Объединённый файл сохранён как merged_terms.xlsx")

Объединённый файл сохранён как merged_terms.xlsx
